In [4]:
%pylab inline
import numpy as np
import scipy as sc
import pandas as pd
from ABC_algorithm import ABC

import statsmodels.api as sm

Populating the interactive namespace from numpy and matplotlib


In [5]:
######
# set up for the normal ABC example
######

prior_mean = -4.0
prior_sd = 3
likelihood_sd = 1

def NormalPriorSampler(n):
    return np.random.normal(loc=prior_mean, scale=prior_sd, size=n)

def NormalLiklihoodSimulator(n, param):
    #unknown mean
    return np.random.normal(loc=param, scale=likelihood_sd, size=n)
    
def NormalSummary(data):
    return np.mean(data, axis=0)

data = np.random.normal(loc=0,scale=likelihood_sd,size=5)

post_sd = np.power((1.0/np.power(prior_sd,2) + len(data)/np.power(likelihood_sd,2)),-1/2)
post_mean = (prior_mean/np.power(prior_sd,2) + np.sum(data)/(np.power(likelihood_sd,2)))/(1/np.power(prior_sd,2) + len(data)/np.power(likelihood_sd,2))
print post_mean, "is posterior mean"
print post_sd, "is posterior std dev"

df = ABC(NormalPriorSampler, NormalLiklihoodSimulator, NormalSummary, 0.2, data , 1000)

-0.857988613704 is posterior mean
0.195652173913 is posterior std dev
